In [ ]:
import pandas as pd
import numpy as np
import numpy
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import math
import random
import shap

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as ltb
import catboost as ctb
from skrebate import ReliefF
from sklearn.model_selection import GroupShuffleSplit

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif, SelectPercentile
from sklearn.metrics import confusion_matrix, classification_report, f1_score, auc, roc_curve, roc_auc_score, precision_score, recall_score, balanced_accuracy_score
from numpy.random import seed
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, KFold, StratifiedKFold, cross_validate
seed(42)
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from tabulate import tabulate

numpy.set_printoptions(threshold=sys.maxsize)

In [ ]:
# Run helper functions
%run Helper.ipynb

In [ ]:
# read the data
df = pd.read_csv('../data/data_ms.csv')

In [ ]:
original_df = df

In [ ]:
response_variable = 'relapse.2y.after.study' # change the response variable
correlated_variables = find_correlated_features(df)

# Baseline Models

In [ ]:
# Filter out baseline features - features mentioned in the paper
df_baseline = df[['nr.relapses.2y.prior.study', 'edss', 'disease.duration', 'treatment.naive.prior.start.cycle',
                  'age', 'nr.Gd.enhanced.lesions', 'months.since.last.relapse', 'gender', 'treatment.during.cycle', 
                  'patient.id',
                  response_variable
                ]]

# Data preprocessing with baseline features
df_baseline, X_train, X_test, Y_train, Y_test, X, Y,  X_train_imputed, X_test_imputed = preprocessing(df_baseline,
                                                                                                      0.25, 
                                                                                                      response_variable,
                                                                                                      isms = True,
                                                                                                      skip_corr = True)


# Train models

xgboost =XGBClassifier(
        use_label_encoder=False,
        eta = 0.02,
        max_depth = 1, 
        max_delta_step = 1,
        subsample = 0.5,
        colsample_bytree = 1,
        tree_method = "auto",
        process_type = "default",
        num_parallel_tree=7,
        objective='multi:softmax',
        min_child_weight = 5,
        booster='gbtree',
        eval_metric = "mlogloss",
        num_class = 2
    )

lgb = ltb.LGBMClassifier(use_missing = True,
                         learning_rate = 0.02,
                         max_depth =2, random_state=0)

catboost = ctb.CatBoostClassifier(iterations=10,
                          learning_rate=0.2,
                          scale_pos_weight=4,
                          loss_function='Logloss',
                          eval_metric='AUC',
                          depth=2)

adaboost = AdaBoostClassifier(random_state=0, learning_rate=0.01, n_estimators=500)

rf = RandomForestClassifier(max_depth=2,
                             n_estimators = np.shape(X_train)[1],
                             criterion = 'entropy', # {“gini”, “entropy”}, default=”gini”
                             class_weight = 'balanced_subsample', # {“balanced”, “balanced_subsample”}, dict or list of dicts, default=None
                             ccp_alpha=0.008,
                             random_state=0)

lr = LogisticRegression(penalty='l2',
                        tol = 1e-3,
                        C=1,
                        l1_ratio = 0.1,
                        class_weight='balanced',
                        random_state=0,
                        solver = 'saga')

models = [xgboost, lgb, catboost, adaboost, rf, lr]

trained_models = init(models, X_train, X_test, X_train_imputed, X_test_imputed, Y_train, Y_test)


# Approach 1 - Without feature selection

In [ ]:
df_app1 = original_df

# data preprocessing
df_app1, X_train, X_test, Y_train, Y_test, X, Y,  X_train_imputed, X_test_imputed = preprocessing(df_app1,
                                                                                                  0.25,
                                                                                                  response_variable,
                                                                                                  isms = True)

# Tune parameters accordingly
xgboost =XGBClassifier(
        use_label_encoder=False,
        eta = 0.1,
        max_depth = 4,
        max_delta_step = 10,
        subsample = 0.5,
        colsample_bytree = 1,
        tree_method = "auto",
        process_type = "default",
        num_parallel_tree=7,
        objective='multi:softmax',
        min_child_weight = 3,
        booster='gbtree',
        eval_metric = "mlogloss",
        alpha  = 0.08,
        num_class = 2
    )

lgb = ltb.LGBMClassifier(use_missing = True, 
                         learning_rate = 0.01, 
                         scale_pos_weight=1,
                         max_depth =4, random_state=0 )

catboost = ctb.CatBoostClassifier(iterations=10,
                          learning_rate=0.1,
                          scale_pos_weight=0.4,
                          depth=3)

adaboost = AdaBoostClassifier(random_state=0,
                              learning_rate=0.1,
                              n_estimators=1000,
                              algorithm = "SAMME.R") 

rf = RandomForestClassifier(max_depth=4,
                             n_estimators = np.shape(X_train)[1] ,
                             criterion = 'gini',
                             class_weight = 'balanced',
                             ccp_alpha=0.01,
                             random_state=0)

lr = LogisticRegression(
    penalty='l2',
    tol = 5e-4,
    C=4,
    l1_ratio = 0.1,
    class_weight='balanced',
    random_state=0,
    solver = 'saga'
)

models = [xgboost, lgb, catboost, adaboost, rf, lr]

trained_models_case1 = init(models, X_train, X_test, X_train_imputed, X_test_imputed, Y_train, Y_test)


# Approach 2 - With feature selection

In [ ]:
# Divide the dataset into two based on sex before feature selection. Use this in sex-stratified models
df_app2 = original_df

# Data preprocessing
df_app2, X_train, X_test, Y_train, Y_test, X, Y,  X_train_imputed, X_test_imputed = preprocessing(df_app2,
                                                                                                  0.25,
                                                                                                  response_variable,
                                                                                                  isms = True)

# Make copy of training and testing data
train_ = X_train.copy()
test_ = X_test.copy()
train_imputed_ = X_train_imputed.copy()
test_imputed_ = X_test_imputed.copy()
Y_train_ = Y_train.copy()
Y_test_ = Y_test.copy()

train_[response_variable] = Y_train_
test_[response_variable] = Y_test_
train_imputed_[response_variable] = Y_train_
test_imputed_[response_variable] = Y_test_

# get sex stratified data
female_X_train, female_X_train_imputed, female_Y_train, female_X_test, female_X_test_imputed, female_Y_test = get_gender_stratified_data(1, train_, test_, 
    train_imputed_, test_imputed_, Y_train_, Y_test_, 'gender')
male_X_train, male_X_train_imputed, male_Y_train, male_X_test, male_X_test_imputed, male_Y_test = get_gender_stratified_data(0,  train_, test_,
    train_imputed_, test_imputed_, Y_train_, Y_test_, 'gender')



In [ ]:
# Feature selection using ReliefF method
best_n = 10 # number of features 

features = get_important_features(best_n, X_train.to_numpy(), Y_train.to_numpy())
display(features)




In [ ]:
# Select columns with selected features
X_train_feat = X_train[features]
X_test_feat = X_test[features]

X_train_feat_imputed_ = X_train_imputed[features]
X_test_feat_imputed_ = X_test_imputed[features]

# Train models. use same models in approach 1
models = [xgboost, lgb, catboost, adaboost, rf, lr]

trained_models_case2 = init(models, X_train_feat, X_test_feat, X_train_feat_imputed_, X_test_feat_imputed_, Y_train, Y_test)


# Approach 3 - Sex-stratified models with selected features

In [ ]:
data = df_app2.copy()
# print gender count in original preprocessed dataset
def get_gender_count(df):
    gender_groupby = df.groupby('gender', as_index=False).agg(total= ('patient.id','count'))
    print(gender_groupby)

get_gender_count(data)

In [ ]:
best_n = 10 # number of features 
features = get_important_features(best_n, female_X_train.to_numpy(), female_Y_train.to_numpy())
print(features)

# feat_names = [
#  'nr.relapses.2y.prior.study',
#  'kfs.7',
#  'kfs.3',
#  'edss',
#  'disease.duration',
#  'age',
#  'lesion_count_3mmfilter',
#  'age.at.diag',
#  'nr.Gd.enhanced.lesions',
#  'months.since.last.relapse']


# Select columns with selected features in sex-stratified data
female_X_train_feat = female_X_train[features]
female_X_test_feat = female_X_test[features]

female_X_train_imputed_feat = female_X_train_imputed[features]
female_X_test_imputed_feat = female_X_test_imputed[features]


In [ ]:
# Train female models

xgboost=XGBClassifier(
        use_label_encoder=False,
        eta = 0.01,#eta between(0.01-0.2)
        max_depth =4, #values between(3-10)
        max_delta_step =8,
        subsample = 0.3,#values between(0.5-1)
        colsample_bytree = 0.9,#values between(0.5-1)
        tree_method = "exact", # {'approx', 'auto', 'exact', 'gpu_hist', 'hist'}
        process_type = "default",
        num_parallel_tree=6,
        objective='multi:softmax',
        min_child_weight = 15,
        booster='gbtree',
        alpha  = 0.08,
        eval_metric = "logloss",
        num_class = 2
    )

lgb = ltb.LGBMClassifier(use_missing = True,
                         learning_rate =0.01,
                         max_depth =3, 
                         random_state=0)

catboost = ctb.CatBoostClassifier(iterations=11,
                          learning_rate=0.085,
                          scale_pos_weight=0.82,
                          eval_metric='AUC',
                          loss_function='Logloss',
                          depth=3)

adaboost = AdaBoostClassifier(random_state=0,
                           algorithm= 'SAMME.R',
                           learning_rate=0.01,
                           n_estimators=150)

rf = RandomForestClassifier(max_depth=5,
                             n_estimators = np.shape(female_X_train_feat)[1],
                             criterion = 'entropy',
                             class_weight = 'balanced_subsample', 
                             ccp_alpha=0.01,
                             random_state=0)

lr = LogisticRegression( penalty='l2',
                        tol =0.01,
                        C=0.1,
                        l1_ratio =0.2,
                        class_weight='balanced',
                        random_state=0,
                        solver = 'saga'
                    )

models = [xgboost, lgb, catboost, adaboost, rf, lr]
    
trained_models_case3_female = init(models, female_X_train_feat, female_X_test_feat,
                                   female_X_train_imputed_feat, female_X_test_imputed_feat, 
                                   female_Y_train, female_Y_test)


In [ ]:

best_n = 10 # number of features 

features = get_important_features(best_n, male_X_train.to_numpy(), male_Y_train.to_numpy())
print(features)

## Below are the selected features
# feat_names = ['nr.relapses.2y.prior.study',
#  'kfs.3',
#  'edss',
#  'disease.duration',
#  'treatment.naive.prior.start.cycle',
#  'age',
#  'age.at.diag',
#  'nr.Gd.enhanced.lesions',
#  'ave.nine.hole.peg.test',
#  'months.since.last.relapse']

male_X_train_feat = male_X_train[features]
male_X_test_feat = male_X_test[features]

male_X_train_imputed_feat = male_X_train_imputed[features]
male_X_test_imputed_feat = male_X_test_imputed[features]

In [ ]:
# Train male models 

xgboost=XGBClassifier(
        use_label_encoder=False,
        eta = 0.02,#eta between(0.01-0.2)
        max_depth =4, #values between(3-10)
        max_delta_step =20,
        subsample = 0.3,#values between(0.5-1)
        colsample_bytree = 0.9,#values between(0.5-1)
        tree_method = "approx", # {'approx', 'auto', 'exact', 'gpu_hist', 'hist'}
        process_type = "default",
        num_parallel_tree=7,
        objective='multi:softmax',
        min_child_weight = 11,
        booster='gbtree',
        alpha  = 0.01,
        eval_metric = "logloss",
        num_class = 2
    )

lgb = ltb.LGBMClassifier(use_missing = True,
                         learning_rate =0.01,
                         max_depth = 1, 
                         random_state=0)

catboost = ctb.CatBoostClassifier(iterations=16,
                                  learning_rate=0.01,
                                  scale_pos_weight=0.95,
                                  eval_metric='Accuracy',
                                  loss_function='Logloss',
                                  depth=3)

adaboost = AdaBoostClassifier(random_state=0,
                           algorithm= 'SAMME.R',
                           learning_rate=0.02,
                           n_estimators=175)

rf = RandomForestClassifier(max_depth=4,
                             n_estimators = np.shape(male_X_train_feat)[1],
                             criterion = 'entropy',
                             class_weight = 'balanced_subsample', 
                             ccp_alpha=0.02,
                             random_state=0)

lr = LogisticRegression(penalty='l2',
                        tol =0.1,
                        C=0.09,
                        l1_ratio = 0.5,
                        class_weight='balanced', 
                        random_state=0,
                        solver = 'saga' 
                    )

models = [xgboost, lgb, catboost, adaboost, rf, lr]

trained_models_case3_male = init(models, male_X_train_feat, male_X_test_feat,
                                   male_X_train_imputed_feat, male_X_test_imputed_feat, 
                                   male_Y_train, male_Y_test)


# SHAP plots

In [ ]:
# Global SHAP plots - FEMALE

# Select the model you wanted to explain
# select XGBClassifier in Approach 2
selected_model = trained_models_case3_female['CatBoostClassifier']

# Calculate shap values
explainer = shap.Explainer(selected_model)
shap_values = explainer(female_X_train_feat)
# shap_values = shap_values[:, :, 0]
shap.summary_plot(shap_values, max_display=20, show=False, plot_size=[15,15])

fig, ax = plt.gcf(), plt.gca()
ax.tick_params(labelsize=18)
ax.set_xlabel("SHAP value (impact on model output)", fontsize=18)
cb_ax = fig.axes[1] 
cb_ax.tick_params(labelsize=18)
cb_ax.set_ylabel("Feature value", fontsize=18)
plt.savefig("figures/MS_FEMALE_Global_SHAP_plot.jpeg" ,bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
# Local SHAP plot - FEMALE

# select the data sample in test data
row_to_show = 84
data_for_prediction = female_X_test_feat.iloc[row_to_show]  # use 1 row of data here. Could use multiple rows if desired
data_for_prediction_array = data_for_prediction.values.reshape(1, -1)

explainer = shap.TreeExplainer(selected_model, check_additivity=False,  feature_perturbation='interventional' )
shap_values = explainer.shap_values(female_X_test_feat.iloc[[row_to_show]])


# Modified the force plot to improve clarity and highlight feature interactions
# Extract the SHAP values for class 0
shap_values_class0 = shap_values[0,]

# Map feature names to corresponding data values
mapping = dict(zip(X_test_feat.columns.tolist(), list(data_for_prediction)))
feature_data = [f"{feature}={mapping[feature]}" for feature in mapping]

# Filter out zero SHAP values and corresponding feature data
sv_ = []
features_ = []
for sv, feature in zip(shap_values_class0, feature_data):
    sv_.append(sv)
    features_.append(feature)

# Sort the features and SHAP values in descending order
sorted_features = [feature for _, feature in sorted(zip(sv_, features_), reverse=True)]
sorted_sv = sorted(sv_, reverse=True)

# Set the threshold to filter out small SHAP values
threshold = 0.00

# Assign colors based on positive or negative SHAP values
colors = ['crimson' if sv >= 0 else 'dodgerblue' for sv in sorted_sv]

# Create the bar plot
fig, ax = plt.subplots(figsize=(16, 10))
ax.tick_params(labelsize=18)
ax.barh(sorted_features, sorted_sv, color=colors, left=threshold)

# Show top values at the top of the plot
ax.invert_yaxis()
plt.xlabel('SHAP value', fontsize=18)
plt.ylabel('Feature value', fontsize=18)
plt.savefig("figures/MS_FEMALE_Local_SHAP_plot.jpeg", bbox_inches='tight', dpi=300)

# Show the plot
plt.show()


In [ ]:
# Global SHAP plots - MALE

# Select the model you wanted to explain
# select XGBClassifier in Approach 2
selected_model = trained_models_case3_male['XGBClassifier']

# Calculate shap values
explainer = shap.Explainer(selected_model, male_X_train_feat)
shap_values = explainer(male_X_train_feat)
shap_values = shap_values[:, :, 1]
shap.summary_plot(shap_values, max_display=20, show=False, plot_size=[15,15])

fig, ax = plt.gcf(), plt.gca()
ax.tick_params(labelsize=18)
ax.set_xlabel("SHAP value (impact on model output)", fontsize=18)
cb_ax = fig.axes[1] 
cb_ax.tick_params(labelsize=18)
cb_ax.set_ylabel("Feature value", fontsize=18)
plt.savefig("figures/MS_MALE_Global_SHAP_plot.jpeg" ,bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
# Local SHAP plot - MALE

# select the data sample in test data
row_to_show = 60
data_for_prediction = male_X_test_feat.iloc[row_to_show]  # use 1 row of data here. Could use multiple rows if desired
data_for_prediction_array = data_for_prediction.values.reshape(1, -1)

explainer = shap.TreeExplainer(selected_model, male_X_test_feat, check_additivity=False,  feature_perturbation='interventional' )
shap_values = explainer.shap_values(male_X_test_feat.iloc[[row_to_show]])

# Modified the force plot to improve clarity and highlight feature interactions
# Extract the SHAP values for class 0
shap_values_class0 = shap_values[1][0,]

# Map feature names to corresponding data values
mapping = dict(zip(X_test_feat.columns.tolist(), list(data_for_prediction)))
feature_data = [f"{feature}={mapping[feature]}" for feature in mapping]

# Filter out zero SHAP values and corresponding feature data
sv_ = []
features_ = []
for sv, feature in zip(shap_values_class0, feature_data):
    sv_.append(sv)
    features_.append(feature)

# Sort the features and SHAP values in descending order
sorted_features = [feature for _, feature in sorted(zip(sv_, features_), reverse=True)]
sorted_sv = sorted(sv_, reverse=True)

# Set the threshold to filter out small SHAP values
threshold = 0.00

# Assign colors based on positive or negative SHAP values
colors = ['crimson' if sv >= 0 else 'dodgerblue' for sv in sorted_sv]

# Create the bar plot
fig, ax = plt.subplots(figsize=(16, 10))
ax.tick_params(labelsize=18)
ax.barh(sorted_features, sorted_sv, color=colors, left=threshold)

# Show top values at the top of the plot
ax.invert_yaxis()
plt.xlabel('SHAP value', fontsize=18)
plt.ylabel('Feature value', fontsize=18)
plt.savefig("figures/MS_MALE_Local_SHAP_plot.jpeg", bbox_inches='tight', dpi=300)

# Show the plot
plt.show()
